In [1]:
!pip install torch transformers pandas numpy matplotlib scikit-learn nltk gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 55.3 MB/s eta 0:00:0000:0100:01


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# reload packages
%load_ext autoreload
%autoreload 2

from transformers import BertTokenizer
import os
import pandas as pd

# Dataset

In [4]:
labels_path = "data/data/en/train-labels-subtask-2.txt"
articles_path = "data/data/en/train-articles-subtask-2/"

In [5]:
import os

os.chdir("../")

In [6]:
import pandas as pd

# Read the dev-labels-subtask-2.txt file
labels_df = pd.read_csv(labels_path, sep="\t")

# Rename the columns for easier processing
labels_df.columns = ["article_id", "frames"]


labels_df.head()

,article_id,frames
0,832959523,"Morality,Security_and_defense,Policy_prescript..."
1,833039623,"Political,Crime_and_punishment,External_regula..."
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq..."
3,814777937,"Political,Morality,Fairness_and_equality,Exter..."
4,821744708,"Policy_prescription_and_evaluation,Political,L..."


In [7]:
# A function to read the article text given its ID
def get_article_content(article_id):
    try:
        with open(f"{articles_path}/article{article_id}.txt", "r") as f:
            return f.read()
    except FileNotFoundError:
        return None

df = labels_df

# Apply the function to get the article content
df["content"] = df["article_id"].apply(get_article_content)

# Drop rows where content could not be found
df.dropna(subset=["content"], inplace=True)

df.head()

,article_id,frames,content
0,832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n\nThose were the time...
1,833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...
3,814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...
4,821744708,"Policy_prescription_and_evaluation,Political,L...",‘Special place in hell’ for those who promoted...


In [8]:
df = df[["article_id", "content"]]

In [9]:
df.head()

,article_id,content
0,832959523,How Theresa May Botched\n\nThose were the time...
1,833039623,Robert Mueller III Rests His Case—Dems NEVER W...
2,833032367,Robert Mueller Not Recommending Any More Indic...
3,814777937,The Far Right Is Trying to Co-opt the Yellow V...
4,821744708,‘Special place in hell’ for those who promoted...


# Customer FrameAxis

In [33]:
import torch
import pandas as pd
from transformers import BertModel, BertTokenizer
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
import numpy as np
from tqdm.notebook import tqdm

In [58]:
class ContextualEmbeddingAnalyzer:
    def __init__(self, antonym_pairs, model_name='bert-base-uncased'):
        self.antonym_pairs = antonym_pairs
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained(model_name)
        if torch.cuda.is_available():
            self.model.cuda()

    def preprocess_content(self, content, remove_stopwords=True):
        # Optionally remove stopwords
        if remove_stopwords:
            stop_words = set(stopwords.words('english'))
            content = ' '.join([word for word in content.split() if word.lower() not in stop_words])
        return content

    def get_word_embeddings_for_sentence(self, sentence):
        # Tokenize and pad the sentence as needed for BERT
        inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {k: v.cuda() for k, v in inputs.items()} if torch.cuda.is_available() else inputs
        
        # Get embeddings from BERT model
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state

    def calculate_difference_vector(self, pos_word, neg_word):
        pos_embedding = self.get_word_embeddings_for_sentence(pos_word)[0][0]
        neg_embedding = self.get_word_embeddings_for_sentence(neg_word)[0][0]
        difference_vector = neg_embedding - pos_embedding
        return difference_vector

    def calculate_cosine_similarities(self, sentence_embeddings, difference_vector):
        cos_similarities = []
        for word_embedding in sentence_embeddings:
            cos_sim = 1 - cosine(difference_vector.cpu().numpy(), word_embedding.cpu().numpy())
            cos_similarities.append(cos_sim)
        return np.mean(cos_similarities)

    def process_row(self, row, remove_stopwords=True):
        article_id = row['article_id']
        content = self.preprocess_content(row['content'], remove_stopwords)
        result_row = {'article_id': article_id}

        # Split sentences and get embeddings once
        sentences = content.split('.')  # Simple split by period to approximate sentence tokenization
        sentence_embeddings_list = [self.get_word_embeddings_for_sentence(sentence) for sentence in sentences]
        
        # Loop over each antonym pair and calculate cosine similarities
        for pos_word, neg_word in tqdm(self.antonym_pairs, desc="Antonym pairs", leave=False):
            difference_vector = self.calculate_difference_vector(pos_word, neg_word)
            article_cos_similarities = []

            for sentence_embeddings in sentence_embeddings_list:
                # Calculate cosine similarities for the sentence
                sentence_cos_sim = self.calculate_cosine_similarities(sentence_embeddings.squeeze(), difference_vector)
                article_cos_similarities.append(sentence_cos_sim)

            # Average cosine similarities across all sentences for this antonym pair
            avg_cos_sim = np.mean(article_cos_similarities) if article_cos_similarities else 0
            result_row[f"{pos_word}_{neg_word}"] = avg_cos_sim

        return result_row

    def process_dataframe(self, df, remove_stopwords=True):
        tqdm.pandas(desc="Analyzing DataFrame")
        result_df = df.progress_apply(lambda row: self.process_row(row, remove_stopwords), axis=1)
        return pd.DataFrame(list(result_df))

In [59]:
df.shape

(432, 2)

In [60]:
# read frameaxis\axes\732_semaxis_axes.tsv into a dataframe
axes_df = pd.read_csv("frameaxis/axes/732_semaxis_axes.tsv", sep="\t", header=None)

# transform the dataframe into tuples list
antonym_pairs = [tuple(x) for x in axes_df.values]

In [61]:
bert_model_path = "../notebooks/models/fine-tuned-model/"

In [ ]:
# Example Usage
analyzer = ContextualEmbeddingAnalyzer(antonym_pairs, model_name=bert_model_path)
result_df = analyzer.process_dataframe(df)

Some weights of the model checkpoint at ../notebooks/models/fine-tuned-model/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../notebooks/models/fine-tuned-model/ and are newly initialized: ['bert.pooler.dense.we

Analyzing DataFrame:   0%|          | 0/432 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

...

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

Antonym pairs:   0%|          | 0/732 [00:00<?, ?it/s]

In [ ]:
result_df

In [57]:
# save the result dataframe to a csv file
result_df.to_csv("../notebooks/models/frameaxis.csv", index=False)